In [1]:
import numpy as np
import os

In [2]:
%cd ../

/Users/eloidieme/dev/python-projects/pr-game-of-life


/Users/eloidieme/dev/python-projects/pr-game-of-life/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
os.getcwd()

'/Users/eloidieme/dev/python-projects/pr-game-of-life'

Règles de mise à jour:
1. Toute cellule morte possédant exactement 3 voisins vivants devient une cellule vivante.
2. Toute cellule vivante possédant 2 ou 3 voisins vivants reste vivante, sinon elle meurt.

In [4]:
grid = np.random.choice([False, True], size=(5, 5))
grid

array([[ True,  True,  True, False, False],
       [ True, False, False, False, False],
       [ True,  True, False, False,  True],
       [False,  True,  True, False, False],
       [ True, False, False, False, False]])

In [5]:
def render(grid):
    s = ''
    s += '-'*(grid.shape[1] + 2)
    s += '\n'

    for i in range(grid.shape[0]):
        s += '|'
        for j in range(grid.shape[1]):
            s += '#' if grid[i, j] else ' '
        s += '|\n'

    s += '-'*(grid.shape[1] + 2)
    s += '\n'

    print(s)

In [6]:
render(grid)

-------
|   ##|
|#    |
|## ##|
|#  ##|
| #  #|
-------



On suppose qu'une cellule au bord de la grille a des voisins sur le bord opposé -> wrapping

In [7]:
coords = []
(grid_height, grid_width) = grid.shape
for i in range(grid_height):
    for j in range(grid_width):
        neighbour_coordinates = [[(k % grid_height, l % grid_width) for k in range(i-1, i+2)] for l in range(j-1, j+2)]
        coords.append(neighbour_coordinates)

In [8]:
def show_neighbours(grid, neighbour_coordinates):
    s = ''
    s += '-'*(grid.shape[1] + 2)
    s += '\n'

    for i in range(grid.shape[0]):
        s += '|'
        for j in range(grid.shape[1]):
            if (i,j) == neighbour_coordinates[1][1]:
                s += 'O'
            elif (i,j) in neighbour_coordinates[0]\
                or (i,j) in neighbour_coordinates[1]\
                or (i,j) in neighbour_coordinates[2]:
                s += '#'
            else:
                s += ' '

        s += '|\n'

    s += '-'*(grid.shape[1] + 2)
    s += '\n'

    print(s)
    

In [9]:
grid = np.array(
    [[False, False, False, False, False],
     [False, False, False, False, False],
     [False, True, True, True, False],
     [False, False, False, False, False],
     [False, False, False, False, False],
    ]
)

In [11]:
show_neighbours(grid, coords[0])

-------
|O#  #|
|##  #|
|     |
|     |
|##  #|
-------



In [12]:
render(grid)

-------
|     |
|     |
| ### |
|     |
|     |
-------



In [16]:
import copy
def update(grid):
    (grid_height, grid_width) = grid.shape
    updated_grid = copy.deepcopy(grid)
    for i in range(grid_height):
        for j in range(grid_width):
            alive_neighbours_count = 0
            for k in range(i-1, i+2):
                for l in range(j-1, j+2):
                    if (k,l) != (i,j) and grid[(k % grid_height, l % grid_width)]:
                        alive_neighbours_count += 1 
            if not grid[i, j] and alive_neighbours_count == 3:
                updated_grid[i, j] = True
            if grid[i, j] and alive_neighbours_count not in [2, 3]:
                updated_grid[i, j] = False
    return updated_grid

In [19]:
render(update(grid))

-------
|     |
|  #  |
|  #  |
|  #  |
|     |
-------



In [18]:
render(update(update(grid)))

-------
|     |
|     |
| ### |
|     |
|     |
-------



In [7]:
from pathlib import Path
import re

file_path = Path('./data/gosper_glider.rle')
try:
    with file_path.open('r') as file:
        grid_lines = [line.strip() for line in file if line.strip()]
except Exception as e:
    raise ValueError(f"Error reading grid file: {file_path}") from e


In [8]:
grid_lines
size_line = grid_lines[0]
pattern_line = ""
for line in grid_lines[1:]:
    pattern_line += line
print(size_line, pattern_line, sep='\n')

x = 36, y = 9, rule = B3/S23
24bo$22bobo$12b2o6b2o12b2o$11bo3bo4b2o12b2o$2o8bo5bo3b2o$2o8bo3bob2o4bobo$10bo5bo7bo$11bo3bo$12b2o!


In [9]:
width_str, height_str, _ = size_line.split(',')
width = int(re.findall(r'[-+]?[0-9]*\.?[0-9]+', width_str)[0])
height = int(re.findall(r'[-+]?[0-9]*\.?[0-9]+', height_str)[0])
width

36

In [10]:
patterns = pattern_line.split('$')

In [11]:
patterns

['24bo',
 '22bobo',
 '12b2o6b2o12b2o',
 '11bo3bo4b2o12b2o',
 '2o8bo5bo3b2o',
 '2o8bo3bob2o4bobo',
 '10bo5bo7bo',
 '11bo3bo',
 '12b2o!']

In [16]:
grid = []

In [17]:
vertical_spacing, horizontal_spacing = 4, 4

In [18]:
total_width = width + 2*horizontal_spacing
for _ in range(vertical_spacing):
    grid.append([0] * total_width)
np.array(grid).shape

(4, 44)

In [22]:
row = [0] * horizontal_spacing
row

[0, 0, 0, 0]

In [23]:
pattern = patterns[0]
row = [0] * horizontal_spacing
nums = "0123456789"
cells = "ob"
running_num = ""
for char in pattern:
    if char in nums:
        running_num += char
    elif char in cells and running_num != "":
        if char == "o":
            for _ in range(int(running_num)):
                row.append(1)
        elif char == "b":
            for _ in range(int(running_num)):
                row.append(0)
        running_num = ""
    else:
        if char == "o":
            row.append(1)
        else:
            row.append(0)
for _ in range(horizontal_spacing):
                row.append(0)
if len(row) < total_width:
    for _ in range(total_width - len(row)):
        row.append(0)

print(np.array(row).shape)

(44,)


In [41]:
for pattern in patterns:
    row = [0] * horizontal_spacing
    nums = "0123456789"
    cells = "ob"
    running_num = ""
    for char in pattern:
        if char in nums:
            running_num += char
        elif char in cells and running_num != "":
            if char == "o":
                for _ in range(int(running_num)):
                    row.append(1)
            elif char == "b":
                for _ in range(int(running_num)):
                    row.append(0)
            running_num = ""
        else:
            if char == "o":
                row.append(1)
            else:
                row.append(0)
    if len(row) < width:
        for _ in range(width - len(row)):
            row.append(0)
    grid.append(row)

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1],
 [1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,


In [ ]:
grid = []
for line_no, line in enumerate(grid_lines, start=1):
    row = []
    if line_no == 0:
        width_str, height_str = line.split(',')
        width = int(re.findall(r'[-+]?[0-9]*\.?[0-9]+', width_str))
        height = int(re.findall(r'[-+]?[0-9]*\.?[0-9]+', height_str))
    else:
        grid_lines = line.split('$')
        grid.append(row)

In [4]:
from GameOfLife.game import Game

grid = Game._parse_grid_from_rle('data/gosper_glider.rle', 4, 4)
grid.shape

(17, 44)